##### Importing new data sets - how to do it

In [8]:
#if you want to save from webpage uncomment this
'''
from bs4 import BeautifulSoup
import urllib.request

def saveFile(webpage, filename):
    websource = urllib.request.urlopen(webpage)
    soup = BeautifulSoup(websource.read(), "html.parser")
    lines = soup.get_text()
    fh = open(filename,"w")
    fh.writelines(lines)
    fh.close() 

#Negative sentiment examples
saveFile(webpage = "https://pythonprogramming.net/static/downloads/short_reviews/negative.txt", 
         filename = "negative.txt")
#Postive sentiment examples
saveFile(webpage = "https://pythonprogramming.net/static/downloads/short_reviews/positive.txt", 
         filename = "positive.txt")
'''

'\nfrom bs4 import BeautifulSoup\nimport urllib.request\n\ndef saveFile(webpage, filename):\n    websource = urllib.request.urlopen(webpage)\n    soup = BeautifulSoup(websource.read(), "html.parser")\n    lines = soup.get_text()\n    fh = open(filename,"w")\n    fh.writelines(lines)\n    fh.close() \n\n#Negative sentiment examples\nsaveFile(webpage = "https://pythonprogramming.net/static/downloads/short_reviews/negative.txt", \n         filename = "negative.txt")\n#Postive sentiment examples\nsaveFile(webpage = "https://pythonprogramming.net/static/downloads/short_reviews/positive.txt", \n         filename = "positive.txt")\n'

In [28]:

short_pos = open("positive.txt", "r").read()
short_neg  = open("negative.txt", "r").read()

documents = []

for line in short_pos.split("\n"):
    documents.append((line, "pos"))

    
for line in short_neg.split("\n"):
    documents.append((line, "neg"))



In [29]:
import nltk

In [30]:
short_pos_words  = nltk.word_tokenize(short_pos)
short_neg_words  = nltk.word_tokenize(short_neg)


In [31]:
len(short_pos_words)
type(short_pos)
len(short_pos)

562832

In [32]:
all_words = []

for w in short_pos_words:
    all_words.append(w.lower())

for w in short_neg_words:
    all_words.append(w.lower())

In [33]:
all_words = nltk.FreqDist(all_words)
#We want to use top 5000 words
word_features = list(all_words.keys())[:5000] 

In [43]:
def find_features(document):
    words = nltk.word_tokenize(document)
    features = {}
    for w in word_features:  #note that word_features contains top 3000 words only(above)
        features[w] = (w in words)
    return(features)

featuresets = [(find_features(doc), category) for (doc, category) in documents]

random.shuffle(featuresets)

In [44]:
training_set = featuresets[:10000] #firs 1900
testing_set  = featuresets[10000:] #rest starting 1900th 

In [45]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
nltk.classify.accuracy(classifier, testing_set)

0.7874015748031497

In [46]:
import pickle
save_classifier = open("naivebayes_pos_neg.pickle", "wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

classifier_f = open("naivebayes_pos_neg.pickle", "rb")
classifier_fromPickle = pickle.load(classifier_f)
classifier_f.close()

In [47]:
##Accuracy
nltk.classify.accuracy(classifier_fromPickle, testing_set)

0.7874015748031497

### Using other classfiers

In [48]:
from nltk.classify.scikitlearn import SklearnClassifier #wrapper of nltk over scikit learn
import pickle

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.preprocessing import LabelEncoder

In [49]:
mnb_classifier = SklearnClassifier(MultinomialNB())
mnb_classifier.train(training_set)
##Accuracy
nltk.classify.accuracy(mnb_classifier, testing_set)

0.8031496062992126

In [50]:
##Some Errors in this 

#gaussian_classifier = SklearnClassifier(GaussianNB())
#gaussian_classifier.train(training_set)
#gaussian_classifier.train(training_set)

##Accuracy
#nltk.classify.accuracy(gaussian_classifier, testing_set)


In [51]:
bernoulli_classifier = SklearnClassifier(BernoulliNB())
bernoulli_classifier.train(training_set)
##Accuracy
nltk.classify.accuracy(bernoulli_classifier, testing_set)

0.8031496062992126

### Other scikitlearn classiers

#LogisticRegression, SGDClassifier
#SVC, LinearSVC, NuSVC


In [52]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

In [53]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
##Accuracy
nltk.classify.accuracy(LogisticRegression_classifier, testing_set)

0.8188976377952756

In [54]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
##Accuracy
nltk.classify.accuracy(SGDClassifier_classifier, testing_set)

/Users/jvsingh/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.8031496062992126

In [55]:
SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
##Accuracy
nltk.classify.accuracy(SVC_classifier, testing_set)

0.6456692913385826

In [56]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
##Accuracy
nltk.classify.accuracy(LinearSVC_classifier, testing_set)

0.7559055118110236

In [57]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
##Accuracy
nltk.classify.accuracy(NuSVC_classifier, testing_set)

0.8110236220472441

#### Note

All of above have their own paramaters which you can customize to imporve the acuuracy

In [58]:
from nltk.classify import ClassifierI
from statistics import mode

In [59]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
        
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return(mode(votes))
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
            
        choice_votes = votes.count(mode(votes))
        conf = choice_votes/len(votes)
        return conf


In [60]:
voted_classfier = VoteClassifier(mnb_classifier,
                                 bernoulli_classifier,
                                 classifier_fromPickle,
                                 SVC_classifier,
                                 NuSVC_classifier
                                )

In [61]:
nltk.classify.accuracy(voted_classfier, testing_set)

0.8110236220472441